<a href="https://colab.research.google.com/github/SantiagoMorenoV/Warehouse-Orders-Fulfillment-Analysis/blob/main/warehouse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Warehouse Orders Fulfillment Analysis**

Warehouse order fulfillment is a critical component of a business's success. Customers expect their orders to be delivered on time and in full. Failure to meet these expectations can result in dissatisfied customers and lost revenue. Inefficient warehouse operations can lead to longer fulfillment times and higher costs. It is essential to analyze warehouse order fulfillment data regularly to identify areas for improvement and optimize operations.

#**Authenticating to GCP**

In [ ]:
from google.colab import auth
auth.authenticate_user()


# **Connecting to the BigQuery API**

Entering the project

In [ ]:
from google.cloud import bigquery
project_id = 'data-cl-4-mod-w3'
client = bigquery.Client(project=project_id)

In [ ]:
print(f"Autenticated in the project '{client.project}'")

Autenticated in the project 'data-cl-4-mod-w3'


# **Accessing the Dataset**

In [ ]:
dataset_ref = client.dataset("warehouse_orders", project='data-cl-4-mod-w3')
dataset = client.get_dataset(dataset_ref)

# **Listing the tables**

In [ ]:
tables = list(client.list_tables(dataset))

for table in tables:
  print(table.table_id)

Orders
Warehouse


# **Checking the table schema**

In [ ]:
table_ref_orders = dataset_ref.table('Orders')
table_orders = client.get_table(table_ref_orders)

In [ ]:
table_orders.schema

[SchemaField('order_id', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('customer_id', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('warehouse_id', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('order_date', 'DATE', 'NULLABLE', None, None, (), None),
 SchemaField('shipper_date', 'DATE', 'NULLABLE', None, None, (), None)]

In [ ]:
table_ref_warehouse = dataset_ref.table('Warehouse')
table_warehouse = client.get_table(table_ref_warehouse)

In [ ]:
table_warehouse.schema

[SchemaField('warehouse_id', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('warehouse_alias', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('maximum_capacity', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('employee_total', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('state', 'STRING', 'NULLABLE', None, None, (), None)]

# **Showing data in dataframes**

### **Orders table**

In [ ]:
client.list_rows(table_orders, max_results=10).to_dataframe()

,order_id,customer_id,warehouse_id,order_date,shipper_date
0,789,3731,8118,2019-01-01,2019-01-04
1,790,3486,8118,2019-01-01,2019-01-04
2,791,2623,8118,2019-01-01,2019-01-04
3,792,9869,8118,2019-01-01,2019-01-04
4,793,6866,8118,2019-01-01,2019-01-04
5,794,8055,8118,2019-01-01,2019-01-04
6,795,1152,8118,2019-01-01,2019-01-04
7,796,5765,8118,2019-01-01,2019-01-04
8,797,6709,8118,2019-01-01,2019-01-04
9,798,4866,2666,2019-01-01,2019-01-04


###**Warehouse table**

In [ ]:
client.list_rows(table_warehouse, max_results=10).to_dataframe()

,warehouse_id,warehouse_alias,maximum_capacity,employee_total,state
0,1543,Somerset Fulfillment Center,210,14,KY
1,2270,Bowling Green Warehouse,280,13,KY
2,9080,Frankfort Fulfillment Center,235,5,KY
3,2666,Lansing Fulfillment Center,290,16,MI
4,3961,Lansing Storage Warehouse,740,22,MI
5,8118,Ann Arbor Fulfillment Center,780,17,MI
6,3417,Gatlinburg Warehouse,620,6,TN
7,4338,Knoxville Fulfillment Center,215,13,TN
8,6509,Memphis Fulfillment Center,755,22,TN
9,9831,Clarsvill Warehouse,400,16,TN


**Select the relevant columns and add a subquery to count the number of orders, then asigning a fulfilment level**

In [ ]:
query = """
SELECT 
  Warehouse.warehouse_id,
  CONCAT(Warehouse.state, ': ', Warehouse.warehouse_alias) AS warehouse_name,
  COUNT(Orders.order_id) AS number_of_orders,
  (SELECT 
    COUNT(*)
    FROM `data-cl-4-mod-w3.warehouse_orders.Orders` Orders)
  AS total_oders,
  CASE 
    WHEN COUNT(Orders.order_id)/(SELECT COUNT(*) FROM `data-cl-4-mod-w3.warehouse_orders.Orders` Orders) <= 0.20 
    THEN "Fulfilled 0-20% of Orders"
    WHEN COUNT(Orders.order_id)/(SELECT COUNT(*) FROM `data-cl-4-mod-w3.warehouse_orders.Orders` Orders) > 0.20 
    AND COUNT(Orders.order_id)/(SELECT COUNT(*) FROM `data-cl-4-mod-w3.warehouse_orders.Orders` Orders) <= 0.60 
    THEN "Fulfilled 21-60% of Orders"
  ELSE "Fulfilled more than 60% of Orders"
  END AS fulfillment_summary
FROM 
  `data-cl-4-mod-w3.warehouse_orders.Warehouse` Warehouse
LEFT JOIN `data-cl-4-mod-w3.warehouse_orders.Orders` Orders
  ON Orders.warehouse_id = Warehouse.warehouse_id
GROUP BY 
  Warehouse.warehouse_id,
  warehouse_name
HAVING 
  COUNT(Orders.order_id) > 0
ORDER BY 
  number_of_orders DESC;
"""

#**Running the query**

In [ ]:
query_job = client.query(query)


# **Waiting for the query to complete and retrieve the results**

In [ ]:
#results = query_job.result()
query_job

QueryJob<project=data-cl-4-mod-w3, location=US, id=3b830171-acde-4807-bdca-9a050eed9454>

#**Printing the results**

In [ ]:
for row in query_job.result():
  print(row)

Row((2666, 'MI: Lansing Fulfillment Center', 3178, 9999, 'Fulfilled 21-60% of Orders'), {'warehouse_id': 0, 'warehouse_name': 1, 'number_of_orders': 2, 'total_oders': 3, 'fulfillment_summary': 4})
Row((8118, 'MI: Ann Arbor Fulfillment Center', 3027, 9999, 'Fulfilled 21-60% of Orders'), {'warehouse_id': 0, 'warehouse_name': 1, 'number_of_orders': 2, 'total_oders': 3, 'fulfillment_summary': 4})
Row((6509, 'TN: Memphis Fulfillment Center', 2403, 9999, 'Fulfilled 21-60% of Orders'), {'warehouse_id': 0, 'warehouse_name': 1, 'number_of_orders': 2, 'total_oders': 3, 'fulfillment_summary': 4})
Row((1543, 'KY: Somerset Fulfillment Center', 548, 9999, 'Fulfilled 0-20% of Orders'), {'warehouse_id': 0, 'warehouse_name': 1, 'number_of_orders': 2, 'total_oders': 3, 'fulfillment_summary': 4})
Row((9080, 'KY: Frankfort Fulfillment Center', 500, 9999, 'Fulfilled 0-20% of Orders'), {'warehouse_id': 0, 'warehouse_name': 1, 'number_of_orders': 2, 'total_oders': 3, 'fulfillment_summary': 4})
Row((4338, 'TN

### Printing the results again, but this time as as dataframe so it will be displayed as a table.

In [ ]:
import pandas as pd

data = [] 
columns = ['warehouse_id', 'warehouse_name', 'number_of_orders', 'total_orders', 'fulfillment_summary']

for row in query_job.result():
  data.append(list(row))

df = pd.DataFrame(data, columns=columns)

print(df)

   warehouse_id                    warehouse_name  number_of_orders  \
0          2666    MI: Lansing Fulfillment Center              3178   
1          8118  MI: Ann Arbor Fulfillment Center              3027   
2          6509    TN: Memphis Fulfillment Center              2403   
3          1543   KY: Somerset Fulfillment Center               548   
4          9080  KY: Frankfort Fulfillment Center               500   
5          4338  TN: Knoxville Fulfillment Center               343   

   total_orders         fulfillment_summary  
0          9999  Fulfilled 21-60% of Orders  
1          9999  Fulfilled 21-60% of Orders  
2          9999  Fulfilled 21-60% of Orders  
3          9999   Fulfilled 0-20% of Orders  
4          9999   Fulfilled 0-20% of Orders  
5          9999   Fulfilled 0-20% of Orders  


I have imported the pandas library to display the results in a table format.

"`data = []`" is for initializing an empty list that will store the results.

After that, I created a loop iterating over the query results. This loop appends each row of the query results to de "`data`" list.

Finally, I implemented a data frame (`df`) object containing data from the query results with the "`columns`" list, and then the results are displayed as a table. 

# **Brief analysis**

This problem had set a goal of 60% or above to be on a good scale of orders fulfilled. From the table above, it is possible to conclude all there would be room for improvement in all the warehouses.

In summary:

*   The Somerset Fulfillment Center in Kentucky has fulfilled the lowest percentage of orders, indicating a need for improvement in its warehouse operations.

*   The Frankfort Fulfillment Center, also located in Kentucky, has a similarly low percentage of fulfilled orders and should also be a focus for improvement.

*   The Knoxville Fulfillment Center in Tennessee is falling behind because of its small percentage of orders fulfilled.

*   The Lansing and the Ann Arbor Fulfillment Centers in Michigan have fulfilled around 30% of orders. Thus, indicating room for improvement in their operations as well.

*   The Memphis Fulfillment Center in Tennessee has fared slightly better, fulfilling around 24% of orders, but still has room for improvement.